In [149]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import time
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn import metrics
from sklearn.impute import SimpleImputer

In [150]:
df=pd.read_csv("adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','captial gain','capital loss','hours per week','native country','Salary'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,captial gain,capital loss,hours per week,native country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [151]:
def Splitting(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return [X_train,y_train,X_test,y_test]

        
class NaiveBayes():
    trainDict=[]
    outCol=[]
    clf=[]
    trainPorb=[]
    y_train=[]
    x_train=[]
    x_test=[]
    y_test=[]
    posOp=[]
    FinalPrediction=[]
    


    def fit(self,x,y):
        X_train=x[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
        y_train=y
        outCol=y.columns
        trainCount=y_train[outCol[0]].value_counts().to_dict()
        col1=list(X_train.columns)
        outputList=y_train[outCol[0]].value_counts().keys().tolist()
        trainDict=dict([(key, []) for key in col1])
        for i in range(0,len(col1)):
            inputList=X_train[col1[i]].value_counts().keys().tolist()
            proxy=dict([(key, dict([(keys, []) for keys in outputList])) for key in inputList])
            trainDict[col1[i]]=proxy
        for i in trainDict.keys():
            for j in trainDict[i].keys():
                for k in trainDict[i][j].keys():
                    num=(X_train.loc[(X_train[i]==j) &  (y_train[outCol[0]]==k)]).shape[0]
                    den=trainCount[k]
                    prob=num/den
                    trainDict[i][j][k]=prob
        X_train=x[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
        clf=GaussianNB()
        clf=clf.fit(X_train,y_train)
        trainProb=(y_train[outCol[0]].value_counts()/y_train.shape[0]).to_dict()
        return self
    
    def predict(self, X):
        X_test=X[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
        outputList=y_train[outCol[0]].value_counts().keys().tolist()
        testDict=dict([(key,dict([(keys, []) for keys in outputList])) for key in X_test.index])
        for i in testDict.keys():
            for j in testDict[i].keys():
                prob=1
                l=0
                for k in trainDict.keys():
                    prob=trainDict[k][X_test.loc[i][l]][j]*prob
                    l=l+1
                testDict[i][j]=prob*trainProb[j]
        X_test=X[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
        s=clf.predict_proba(X_test)        
        r=0
        c=0
        prediction=dict([(key,dict([(keys, []) for keys in posOp])) for key in y_test.index])
        for i in prediction.keys():
            c=0
            for j in prediction[i].keys():
                prediction[i][j]=s[r][c]
                c=c+1
            r=r+1
        testInd=X_test.index
        FinalPrediction=dict([(keys,[]) for keys in testInd])
        for i in FinalPrediction.keys():
            maxi=0
            pr=''
            p=0
            proba=[]
            for j in posOp:
                p=prediction[i][j]*testDict[i][j]
                if(p>maxi):
                    maxi=p
                    pr=j
                    proba=ProbOp[j]
            FinalPrediction[i]=pr*proba
        return FinalPrediction
    
    def score(self,X,y):
        from sklearn.metrics import accuracy_score
        return accuracy_score(y,FinalPrediction)



In [153]:
y=df[['Salary']]
X=df[[col for col in df.columns if col!='Salary']]
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X=pd.DataFrame(data=imp_mode.fit_transform(X),index=X.index,columns=[col for col in df.columns if col!='Salary'])
[X_train,y_train,X_test,y_test]=Splitting(X,y)
posOp=y_train['Salary'].value_counts().keys().tolist()
Probop=dict([(key,[]) for key in posOp])
for i in Probop.keys():
        Probop[i]=(y_test['Salary'].value_counts()[i])/y_test.shape[0]

In [154]:
nb=NaiveBayes()
nb=nb.fit(X_train,y_train)

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [155]:
predict=nb.predict(X_test)
# score=nb.score(X_test,y_test)

NameError: name 'trainDict' is not defined